In [2]:
from pyspark.sql import SparkSession

In [ ]:
spark = SparkSession.builder.appName("DesafioFinalVendas").getOrCreate()

25/04/04 23:43:16 WARN Utils: Your hostname, codespaces-a65c79 resolves to a loopback address: 127.0.0.1; using 10.0.4.17 instead (on interface eth0)
25/04/04 23:43:16 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/04/04 23:43:17 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


25/04/04 23:43:33 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [7]:
# Carregar os dados de vendas a partir de um arquivo CSV
df_vendas = spark.read.csv("./dados/vendas.csv", header=True, inferSchema=True)

# Exibir as primeiras 5 linhas para entender a estrutura dos dados
df_vendas.show(5)


+----------+------------+-----------+
|id_cliente|valor_compra|data_compra|
+----------+------------+-----------+
|         1|      150.75| 2023-01-15|
|         2|       230.5| 2023-02-20|
|         3|       100.0| 2023-03-10|
|         4|      450.25| 2023-04-05|
|         5|       320.0| 2023-05-22|
+----------+------------+-----------+
only showing top 5 rows



In [8]:
from pyspark.sql.functions import sum

# Agrupar os dados por 'id_cliente' e calcular o valor total de compra de cada cliente
df_maior_compra = df_vendas.groupBy("id_cliente").agg(sum("valor_compra").alias("total_compra"))

# Ordenar os clientes pelo valor total da compra em ordem decrescente
df_maior_compra_ordenado = df_maior_compra.orderBy("total_compra", ascending=False)

# Exibir os 5 clientes com maior valor de compra
df_maior_compra_ordenado.show(5)


+----------+------------+
|id_cliente|total_compra|
+----------+------------+
|        10|      1000.0|
|         8|       700.5|
|         6|      550.75|
|         4|      450.25|
|         5|       320.0|
+----------+------------+
only showing top 5 rows



In [9]:
from pyspark.sql.functions import year

# Extrair o ano da coluna 'data_compra'
df_vendas_com_ano = df_vendas.withColumn("ano", year("data_compra"))

# Agrupar por 'ano' e calcular o total de vendas por ano
df_vendas_anuais = df_vendas_com_ano.groupBy("ano").agg(sum("valor_compra").alias("total_vendas_ano"))

# Exibir o total de vendas por ano
df_vendas_anuais.show()


+----+----------------+
| ano|total_vendas_ano|
+----+----------------+
|2023|         3833.75|
+----+----------------+



In [10]:
# Salvar os resultados como CSV
df_maior_compra_ordenado.write.csv("maior_compra_clientes.csv", header=True)
df_vendas_anuais.write.csv("total_vendas_anuais.csv", header=True)


In [11]:
# Salvar os resultados como JSON
df_maior_compra_ordenado.write.json("maior_compra_clientes.json")
df_vendas_anuais.write.json("total_vendas_anuais.json")
